# Introduction to Stochastic Gradient Descent
## May, 2024
## A hands-on notebook by Fariman.AA and Kiani.M


### Introduction, Whats gradient descent algorithm?


### Stochastic gradient descent (SGD)

### Why we use Stochastic gradient descent (SGD)?

### Stochastic gradient descent algorithm implementation in Python



In this section of the notebook, we present an implementation of the Stochastic Gradient Descent (SGD) algorithm in Python. The implementation leverages the NumPy library, so first we import NumPy.

In [1]:
import numpy as np

We define few hyperparameters that influence the SGD optimization process.

* learning_rate: This parameter controls the step size taken in each iteration.
* epochs: This parameter specifies the number iterations.
* batch_size: This parameter determines the number of data samples used for updating values in each iteration.
* tolerance: This parameter defines the convergence criterion .
* weights: This variable represents the weight associated with the model $ (y = weight * x + bias) $.
* bias: This variable represents the bias associated with the model $ (y = weight * x + bias) $ .

In [13]:
learning_rate = 0.01
epochs = 1000
batch_size = 32
tolerance = 1e-3
weights = None
bias = None

We proceed by defining a predict function. This function takes an input vector, denoted by X, and returns a predicted output value, denoted by Y. The prediction is computed using the linear regression equation Y = w * X + b, where w represents the weight vector and b represents the bias.

In [29]:
# Linear regression equation, y = w * x + b
def calculate(X):
  return np.dot(X,weights) + bias

We now define the cost function, a significant advantage of SGD lies in its flexibility regarding the choice of cost function. The code can adapt to various cost functions. In this instance, we have opted for the Mean Squared Error (MSE) cost function.

In [19]:
# Cost function
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

Lets define our gradient function. The gradient function calculates the gradients of the cost function (which we'll assume is MSE in this case) with respect to the weights (w) and the bias (b) for a given batch of data (X_batch, y_batch).

In [20]:
# Return gradient_weights (partial derivative with respect to weights) and
# griadient_bias (partial derivative with respect to bias)
def gradient(X_batch, y_batch):
    y_pred = calculate(X_batch)
    error = y_pred - y_batch
    gradient_weights = np.dot(X_batch.T, error) / X_batch.shape[0]
    # we dont use mean_square_error function here for calculatin gradient bias
    # becuse we have error defiend earlier for returning gradient_weights
    gradient_bias = np.mean(error)
    return gradient_weights, gradient_bias

We now introduce the SGD function, this function follows an structure:
* Initialization: The function begins by initializing the weights (w) and bias term (b) with random values.
* Iteration: The core of the SGD function lies within the iterative loop.
* Random batch selection: Within each iteration, a random subset of data points (N) is selected from the dataset.

* Gradient update: This function is utilized to calculate the gradients of the cost function with respect to the weights and bias term for the current batch.

* Weight and Bias Update: This function updates the weights and bias term according cost function minimization.

* Termination: The iteration loop terminates when the maximum number of epochs has been reached or convergence criterion is satisfied.


In [33]:
def SGD(X, y):
    n_samples, n_features = X.shape
    global weights
    weights = np.random.randn(n_features)
    global bias
    bias = np.random.randn()

    for epoch in range(epochs):
      # np.random.permutation randomly permute a sequence
      indices = np.random.permutation(n_samples)
      X_shuffled = X[indices]
      y_shuffled = y[indices]

      for i in range(0, n_samples, batch_size):
          X_batch = X_shuffled[i:i+batch_size]
          y_batch = y_shuffled[i:i+batch_size]

          gradient_weights, gradient_bias = gradient(X_batch, y_batch)
          weights -= learning_rate * gradient_weights
          bias -= learning_rate * gradient_bias

      if epoch % 100 == 0:
        y_pred = calculate(X)
        loss = mean_squared_error(y, y_pred)
        print(f"Epoch {epoch}: Loss {loss}")

      if np.linalg.norm(gradient_weights) < tolerance:
          print("Convergence reached.")
          break

    return weights, bias

### Lets review a simple example of SGD

In [32]:
# Create random dataset with 100 rows and 5 columns
X = np.random.randn(100, 5)
# create corresponding target value by adding random
# noise in the dataset
y = np.dot(X, np.array([1, 2, 3, 4, 5]))\
    + np.random.randn(100) * 0.1

w,b = SGD(X,y)
print('Best Weight:' + str(w))
print('Best Bias:' + str(b))
# Predict using predict method from model
y_pred = w*X+b
#y_pred

Best Weight:[1.01221837 1.99274418 2.98478503 3.98803252 4.98904489]
Best Bias:-0.0009756412032080085
